# Assignment 3

This notebook is the solution to HW3, written by Yaniv Bin and Tair Hakman.

We first would like to import all the required modules in order for our code to run properly:

In [1]:
import nltk, re, itertools
import matplotlib.pyplot as plt
import numpy as np
from nltk import nonterminals, Nonterminal, Production, induce_pcfg
from nltk.parse import generate
from nltk.grammar import Nonterminal, Production, toy_pcfg2
from nltk.probability import DictionaryProbDist
from nltk import Tree, CFG, PCFG, Nonterminal
from nltk.treetransforms import chomsky_normal_form
from numpy import log
from nltk.corpus import LazyCorpusLoader, BracketParseCorpusReader
from collections import *

Now after doing so we can go ahead and solve the tasks.

## Part 1

In the first part of the assignment we will discuss designing CFG for NLP task.
<br> We were given the following code which read CFGs from string representation, and parse sentences given a CFG using a  a bottom-up parsing algorithm:

In [ ]:
sg = """
S -> NP VP
VP -> IV | TV NP
NP -> 'John' | "bread"
IV -> 'left'
TV -> 'eats'
"""
g = CFG.fromstring(sg)

# Bottom-up  parser
sr_parser = nltk.ShiftReduceParser(g, trace=2)

# Parse sentences and observe the behavior of the parser
def parse_sentence(sent):
    tokens = sent.split()
    trees = sr_parser.parse(tokens)
    for tree in trees:
        print(tree)

parse_sentence("John left")
parse_sentence("John eats bread")

### Part 1.1

First, an important note about our parser, the Shift-Reduce parser. This parser does not support ambiguity at all. If multiple reductions are available for a single word, it will simply choose the first reduction listed (and if it fails, it will not go back to try the second reduction). This means we can't have the same NT on the rhs of two different rules.

We're required to support a new list of sentences. In order to explain the process of creating the CFG, we would like to split the sentence list into three different lists (slightly altering the original sentence order). The first list:

John left
John eats bread
John loves Mary
She loves John
She loves them 
Everybody loves John
A boy loves Mary

And here's the grammar:

In [ ]:
sg = """
S -> DET NP VP | NP VP | PR_NOM VP
NP -> 'John' | 'bread' | 'Mary' | 'boy'
IV -> 'left' 
VP -> IV | TV NP | TV PR_ACC
TV -> 'eats' | 'loves'
PR_NOM -> 'She' | 'Everybody'
PR_ACC -> 'them'
DET -> 'A'
"""

g = CFG.fromstring(sg)

# Bottom-up  parser
sr_parser = nltk.ShiftReduceParser(g, trace=2)

sentences = """John left
John eats bread
John loves Mary
She loves John
She loves them 
Everybody loves John
A boy loves Mary
"""

sentences = sentences.split("\n")

for i in range(2,7):
    parse_sentence(sentences[i])
    print("---------")

This bit was relatively easy to deal with. We've introduced pronouns, who are split (so far) into two categories by their case: Nominative (He, she etc) and accusative (her, them) etc. Nominative pronouns appear in the beginning of sentences, accusative ones in the end. So we've added pronouns as an alternative to NPs in the appropriate places. We've also introduced the category of determiners (only "A" so far), which can start a sentence before a noun ("A book") - note that a determiner can't precede a pronoun ("A she" is not legal).

Our next sentences are a bit different:

They love Mary 
They love her

The difference is the use of 'love' instead of 'loves'. This is a different kind of verb that follows different speakers - "They love" and "He loves" are legal, but "They loves" and "He love" aren't. So we'll split our grammar into two categories - sentences of type 1 (without s), and type 2.

In English, the category 1 is I/we/you/they (in this case, they will all be followed by 'love'), and category 2 is he/she/it (all followed by 'loves'). This separation is somewhat similar to singular/plural, however it is not the same - note that "I" (singluar) is in the same category as "we" and "they" (plural), while "you" can refer to both singluar and plural. So for a lack of better name, we will refer to those categories as categories 1 and 2.

So here's our new grammar:


In [ ]:
#1 = I, we, you, they
#2 = He, she, it

sg = """
S -> S1 | S2
S1 -> PR_NOM1 VP1
VP1 -> TV1 NP | TV1 PR_ACC
TV1 -> 'love'
PR_NOM1 -> 'They'

S2 -> DET NP VP2 | NP VP2 | PR_NOM2 VP2
VP2 -> IV | TV2 NP | TV2 PR_ACC
TV2 -> 'eats' | 'loves'
PR_NOM2 -> 'She' | 'Everybody'

NP -> 'John' | 'bread' | 'Mary' | 'boy'
IV -> 'left' 
PR_ACC -> 'them' | 'her'
DET -> 'A'
"""

g = CFG.fromstring(sg)

# Bottom-up  parser
sr_parser = nltk.ShiftReduceParser(g, trace=2)

sentences = """John left
John eats bread
John loves Mary
She loves John
She loves them 
Everybody loves John
A boy loves Mary
They love Mary 
They love her
"""

sentences = sentences.split("\n")

for i in range(7,9):
    parse_sentence(sentences[i])
    print("---------")

As you can see, we split most rules into types 1 and 2. Note that accusative pronounes aren't changed - the word "her" will be the same in both "I love her" and "she loves her".
Our last group of sentences is:

John gave Mary a heavy book
John gave it to Mary

This is when we encounter the limitations of the SRP. Looking at the first sentence, "John gave Mary" is already a legal sentence in our grammar (assuming the verb "gave" is added). So once we parse "John gave Mary", before the parser continues to the next word, those words will be reduced to "S". The best solution would be to use a better, less limited parser that supports ambiguity. But sticking with the SR parser, our solution will be a bit untidy - we will simply add a rule that adds the rest of the words after S.

In [ ]:
sg = """
S -> S1 | S2 | S DET AD_NP | S PREP_NP
S1 -> PR_NOM1 VP1
VP1 -> TV1 NP | TV1 PR_ACC
TV1 -> 'love'
PR_NOM1 -> 'They'

S2 -> DET NP VP2 | NP VP2 | PR_NOM2 VP2 | 
VP2 -> IV2 | TV2 NP | TV2 PR_ACC
TV2 -> 'eats' | 'loves' | 'gave'
PR_NOM2 -> 'She' | 'Everybody'
IV2 -> 'left'

NP -> 'John' | 'bread' | 'Mary' | 'boy' | 'book'
AD_NP -> ADJ NP
PREP_NP -> PREP NP
PR_ACC -> 'them' | 'her' | 'it'
PREP -> "to"
DET -> 'A' | 'a'
ADJ -> 'heavy'
"""

g = CFG.fromstring(sg)

# Bottom-up  parser
sr_parser = nltk.ShiftReduceParser(g, trace=2)

Note: the use of infinite recursion causes NLTK to throw some warnings - however they are false, as all sentences are parsed (and clearly all roles are used).

Let's verify all 11 sentences are parsed correctly:

In [ ]:
sentences = """John left
John eats bread
John loves Mary
She loves John
She loves them 
Everybody loves John
A boy loves Mary
They love Mary 
They love her
John gave Mary a heavy book
John gave it to Mary"""

'''
Number: singular / plural (e.g., he/they)
Gender: masculine / feminine / neutral (e.g., he/she/it)
Case: nominative / accusative (e.g., he/him)
'''

sentences = sentences.split("\n")

for i in range(11):
    parse_sentence(sentences[i])
    print("---------")

As for gender: we didn't need to encode it, because gender doesn't make a grammatical difference in English. Most pronouns (I/we/you/they) are netural to gender. And he/she, the only pronouns who are specific to a gender, behave the same gramatically ("he loves her", "she loves her"). The same also goes for nouns who have a clear gender ("John loves her", "Mary loves her").

#### Part 1.1.2

Our grammar isn't perfect, and it does overgenerate in certain cases. One weakness is the lack of separation between different nouns, even though certain nouns can't logically do certain actions. This allows us to parse illogical sentences, such as:

In [ ]:
parse_sentence("bread gave book a heavy Mary")

Another weakness is the workaround we've added for the last sentences, which basically allows us to add certain combinations (such as DET ADJ NP) at the end of every sentence:

In [ ]:
parse_sentence("John loves Mary a heavy book")

And since the role is recursive, we can add it as many times as we like:

In [ ]:
parse_sentence("John loves Mary a heavy book a heavy book a heavy book a heavy book a heavy book a heavy book a heavy book")

### Part 1.2

So now we're expected to add support to these sentences:

John saw a man with a telescope
John saw a man on the hill with a telescope

Mary knows men and women
Mary knows men, children and women

John and Mary eat bread
John and Mary eat bread with cheese

We can keep using the same trick from the end of last question - a recursive role to allow us to add certain suffixes at the end of a legal sentence (potentially endlessley). For example: if "John saw a man" is a legal sentence in our language, we can add two possible suffixes after it: "on the hill" and "with a telescope". This will make all combinations legal - many of them makes sense ("John saw a man with a telescope" "...man on the hill" "...man on the hill with a telescope" "...man with a telescope on the hill"), but also, as we've seen in the last question, we're exposed to infinite loops.

Here is the new grammar:

In [ ]:
sg = """
S -> S1 | S2 | S DET AD_NP | S PREP_NP | S PREP DET_NP | S CONJ_NP
S1 -> PR_NOM1 VP1
VP1 -> TV1 NP | TV1 PR_ACC
TV1 -> 'love' | 'eat'
PR_NOM1 -> 'They' | NP CONJ NP

S2 -> DET NP VP2 | NP VP2 | PR_NOM2 VP2 | 
VP2 -> IV2 | TV2 NP | TV2 PR_ACC | TV2 DET_NP
TV2 -> 'eats' | 'loves' | 'gave' | 'saw' | 'knows'
PR_NOM2 -> 'She' | 'Everybody'
IV2 -> 'left'

NP -> 'John' | 'bread' | 'Mary' | 'boy' | 'book' | 'man' | 'telescope' | 'hill' | 'men' | 'women' | 'children' | 'cheese'
AD_NP -> ADJ NP
PREP_NP -> PREP NP
DET_NP -> DET NP
CONJ_NP -> CONJ NP
PR_ACC -> 'them' | 'her' | 'it'
PREP -> "to" | 'with' | 'on'
DET -> 'A' | 'a' | 'the'
CONJ -> 'and' | ','
ADJ -> 'heavy'
"""

g = CFG.fromstring(sg)

# Bottom-up  parser
sr_parser = nltk.ShiftReduceParser(g, trace=2)

# Parse sentences and observe the behavior of the parser
def parse_sentence(sent):
    tokens = re.findall(r"[\w']+|[.,!?;]", sent)
    trees = sr_parser.parse(tokens)
    for tree in trees:
        print(tree)

sentences = """John saw a man with a telescope
John saw a man on the hill with a telescope
Mary knows men and women
Mary knows men, children and women
John and Mary eat bread
John and Mary eat bread with cheese"""


sentences = sentences.split("\n")

for i in range(6):
    parse_sentence(sentences[i])
    print ("---------")


A few specific notes about implementation:

-We've added the category of conjuction, such as 'and'.
-The 4th sentence includes a comma, which also has a grammatical role. However, the split function previously used does not split punctuation (it created the word 'men,' rather than 'men' and ','), so we've changed the split method to a method that will split the coma (using regular expressions). The comma is treated as a conjuction word, similar to 'and' - again, the endless recursion helps us to add two suffixes (", children" "and women") on top of "Mary knows".
-The expression "John and Mary" is equivalent to "they", so it's treated as a nominative pronoun.


#### Part 1.2.2

Looking at the given examples and the number of coordination:
"John and Mary" - 2 people
"John or Mary" - 1 people
"John or the children" - either one or an unspecified (bigger than 1) number of people.

The first example was already dealt with in the last question - by treating "NP and NP" as equivalent to "They".
The case of 'or' is a bit more complicated, because it can refer to either singular or plural, as the last two examples show. One of the weaknesses of our grammar (as we demonstrate in the next question), is not separating singluar and plural nouns. So a possible solution to deal with the problem is to separate the category NP into NP_PL and NP_SIN (plural and singular). After we split it, we can safely treat "NP_PL or NP_PL" as plural (similar to "they"), and "NP_SIN or NP_SIN" as singluar (similar to he/she). 

The mixed case, "NP_PL or NP_SIN" is still ambiguous - but this is not a problem with our grammar, but with the English language. When we say 'John or the children', we don't know whether we're talking about singluar or plural. The correct grammar in those cases is the singluar grammar - "John or the children love him" (rather than "loves"), so we can treat the mix case as similar to 'They'.

#### Part 1.2.3

Neither of the issues we pointed out on the previous question are now fixed. We're still prone to illogical nouns use ("telescope eats a children with boy on a John" parses ok) and still prone to endlessley stacking sentences on top of each other. 
Some of our new added NPs ("women", "men") are plural - meaning swapping them with a singluar NP is problematic. For example, 'Mary knows women' makes sense, but "Mary knows hill" is gramatically wrong - we would have needed a determiner ("Mary knows **a** hill") to create a correct sentence.

In [ ]:
parse_sentence ("Mary knows hill")

A new problem is introduced with conjuction - our conjuction role treats the comma sign and the word 'and' in the same way. Which means we can create a list with any combination of those. Of course, English grammar demands each item on the list to be comma separated, while "and" only appears before the last item ("a, b, c and d"). This example breaks both roles:

In [ ]:
parse_sentence ("Mary knows men and children and book and telescope and hill, boy, John")

## Part 2

### Part 2.1

NLTK has a model called generate which is able to generate sentences given a CFG grammer. 
Our goal in this part is to create a generator for a PCFG instead of a CFG.

So lets start by writing our generator function, which, given a PCFG, return a tree representing the generation process:

In [ ]:
# generates a tree from a PCFG grammer 
def pcfg_generate(grammar):
    start = grammar.start()
    return(pcfg_generate_root(grammar, start))
    
# generates a tree from a given root based on the PCFG grammer    
def pcfg_generate_root(grammar, root):
    #if it's not a terminal it means we have to generate from the probabilities
    if isinstance(root, Nonterminal):
        item_productions = grammar.productions(lhs=root)
        dict = {}
        for pr in item_productions: dict[pr.rhs()] = pr.prob()
        item_probDist = DictionaryProbDist(dict)
        prod = item_probDist.generate()
        if (len(prod) == 2):
            lh = prod[0]
            rh = prod[1]
            lh_tree = pcfg_generate_root(grammar, lh)
            rh_tree = pcfg_generate_root(grammar, rh)
            return Tree(root, [lh_tree, rh_tree])
        else:
            lh = prod[0]
            return Tree(root,[pcfg_generate_root(grammar, lh)])
           
    #if it's a terminal we can just return it
    else:
        return root

Now we can test our function with the *toy_pcfg2* grammer:

In [ ]:
example_tree = pcfg_generate(toy_pcfg2)
Tree.fromstring(example_tree.pformat()).pretty_print()

As we can see, not all the resulting sentences makes sense... This might be due to the fact the PCFG is based on distribution, and therefore it will always lean towards certain phrases. 

We will now continue on to do some validations in the next subsections - 

#### Part 2.1.1

The first thing we are going to do is to generate 1000 random trees given the *toy_pcfg2* grammer, and save all resulting trees into a file called "toy_pcfg2.gen"

In [ ]:
def create_trees_file(n=1000):
    for i in range(n):
        current_tree = pcfg_generate(toy_pcfg2)
        with open("toy_pcfg2.gen", "a+") as f:
            current_tree.pprint(stream=f)
            f.write("*")

In [ ]:
create_trees_file(n=1000)

Now once we created such file we can use it to conduct some experiments on the resulting trees.

#### Part 2.1.2

The first thing we need to do is to read our trees from our file:

In [ ]:
read_file = open("toy_pcfg2.gen", "r")
buffer = ""
for line in read_file :
    buffer += line
trees = buffer.split("*")
trees = trees[:len(trees) - 1]

Now that we have our trees, we would like to compute the frequency distribution of each non-terminal and pre-terminal in the generated corpus - to do so the first thing we're going to need is the tree's productions:

In [ ]:
toy_sample_productions = []
for tree in trees:
    toy_sample_productions += Tree.fromstring(tree).productions()

Once we have those we can calculate the distribution of each non-terminal and pre-terminal, because it's the same as the productions distributions.

When using NLTK we found there are two ways of doing so, one is using native functions in the NLTK library, and the other is to implement it by ourselves. We will demonstrate both and use that fact to evaluate our own function.

The first way, using NLTK native function, is as follows:

In [ ]:
#One way of achieving our goal
pcfg_induced = induce_pcfg(Nonterminal('S'), toy_sample_productions).productions()

The second way is to use ConditionalFreqDist for every left-hand and right-hand side of the production, and then create a ConditionalProbDist using MLE (like we did in the previous assignemnts), this will also result in the distribution.

In [ ]:
def calc_productions_probabilities(productions):
    #And another way
    cfd = nltk.ConditionalFreqDist((production.lhs(), production.rhs()) for production in productions)
    cpd_mle = nltk.ConditionalProbDist(cfd, nltk.MLEProbDist)
    return cpd_mle

now if we compare the distributions from both methods:

In [ ]:
toy_sample_probability = calc_productions_probabilities(toy_sample_productions)

In [ ]:
print("First Method:")
for nt in pcfg_induced:
    print(nt)
print("\nSecond Method:")
for cond in toy_sample_probability:
    for sample in toy_sample_probability[cond].samples():
        print("{} -> {} {}".format(cond, sample, toy_sample_probability[cond].prob(sample)))

And as we can see the results are the same (give or take rounding).

After we calculated the distribution for our toy sample, we can calculate them for the entire **toy_pcfg2** grammar, we will use the results in the next section:

In [ ]:
toy_pcfg_probability = calc_productions_probabilities(toy_pcfg2.productions())

#### Part 2.1.3

What we would like to calculate next is a measure called **KL Divergence**.
<br>The *Kullback–Leibler divergence*(also called relative entropy) is a measure for how much one probability distribution differ from another.
<br> In order to do so we will consider two distributions,
<br> $P = (x_1: p_1, x_2: p_2, \dots , x_n: p_n)$ and $Q = (y_1: q_1, y_2: q_2, \dots, y_n: q_n)$ such that $\Sigma{p_i} = 1$ and $\Sigma{q_i} = 1$
<br>The KL Divergence is then defined as:
<br>$KL(P,Q) = \Sigma_{i=1, \dots, n} {p_i * \log{\frac{p_i}{q_i}}}$

One can notice this calculation is somewhat problematic because it assumes some things that doesn't always hold in real world:
- Both $P$ and $Q$ are defined over the same outcomes $x_i$ 
- We never face $q_i = 0$ for some $i$ or $p_i = 0$ for some $i$

One way of facing that is to decide $0 * \log{0} = 0$ but this only help when $p_i = 0$ or both $p_i = 0$ and $q_i = 0$, so in the case when $p_i \neq 0$ and $q_i = 0$ we define the difference as infinite. 

This method of dealing with those issue might work in some cases and calculations, but in our case $P$ and $Q$ are derived from observations and sample counting - that is, $P$ and $Q$ are probability distributions derived from frequency distributions. We have to therfore take into account some things that might happen:
- there is a big chance for unseen events - meaning samples that appear in one probability but not the other.
- Because of that we can't define the difference as infinite.

In order to deal with that issue, we have to implement a *smoothing* technique on the measure.

A quick way of applying a smoothing over the measure is to use a method called *absolute discounting*.
<br> Absolute discounting would work in our case as follows:
- We define a small cconstant $\epsilon$ (for example $\epsilon = 0.0001$)
- Define $SP = {a, b, c}$ the samples observed in $P$
- Define $CP = |SP| = 3$, the number of samples observed in $P$
- Similarly, $SQ = {a, b, d}$ and $CQ = 3$
- Define $SU = SP U SQ = {a, b, c, d}$ - all the observed samples and $CU = |SU| = 4$

After we define all of this terms, we can them modify our probabilities (smooth) as follows:
- let $pc = \epsilon * \frac{|SU-SP|}{|SP|}$ and $qc = \epsilon * \frac{|SU-SQ|}{|SQ|}$
- for each $p_i$ :
 - let $p'_i = p_i - pc$ if $i\in{SP}$
 - let $p'_i = \epsilon$ otherwise
- similarly for $q'_i$ with $qc$

After we do so, we never get a 0 probability because each probability is defined by it's relative part in the sample(smoothing). and so now we can calculate $KL(P', Q')$ without getting infinite difference.

After defining the smoothing technique we can go ahead and implement the function in python:

In [ ]:
def compute_kl_divergence(p, q, eps=0.0001):
    sp = list(p.samples())
    cp = len(sp)
    sq = list(q.samples())
    cq = len(sq)
    su = set(sp + sq)
    cu = len(su)
    pc = eps * ((cq) / float(cp))
    qc = eps * ((cp) / float(cq))
    kl = 0.0
    for i in su:
        if i in sp:
             p_i = p.prob(i) - pc
        else:
            p_i = eps
        if i in sq:
            q_i = q.prob(i) - qc
        else:
            q_i = eps
            
        kl +=  p_i * log(p_i / float(q_i))
    return kl

And now we would like to apply it to our two probability distributions (one is our toy sample and one is the original toy test grammar):

In [ ]:
divergence = {}
for condition in toy_sample_probability.conditions():
    kl = compute_kl_divergence(toy_sample_probability[condition], toy_pcfg_probability[condition])
    divergence[condition] = kl
print("Divergence:", divergence)

We will analyze the results in the next section. 

#### Part 2.1.4

As we've just seen, the results are similar on some of the tags (for example S and PP), and divert on others. 
The similar rules (with 0 diversion) are similar due to the fact that in the original toy grammar, this rules have only one derivation rule and it is derived with a 1.0 probabilty, and so our samples will all use that rule, the diversion happens once we look at rules with more "options", then in our samples such rules won't exist at all (because our sampler is based on the probabilities and is limited in the size it generates - just 1000 trees).

<TODO> add more here

### Part 2.2

What we'd like to do now is to explore how we can induce a PCFG from a given treebank.
<br>To do so we would use the Penn Treebank given by NLTK. 

#### Part 2.2.1

The first thing we'd like to do is to induce the PCFG from the original treebank without transforming it into a CNF.

The Penn Treebank comes with some additional information for the tags, for example some NP tags has additional information like NP-something(change this) , we don't need this information in our induction(it actually might mess up the results because each NP tag then will be considered seperately and this will mess the probabilities). 
<br>Another thing it holds is some NONE tags, used for "missing" words, for example a sentence like:

We don't want to take NONE tags into account in our induced PCFG, so a suggested method is to ignore all NONE tags (just remove their branch from the tree).

Lets implement a function that simplifies a tag then:

In [ ]:
def simplify_functional_tag(tag):
    if '-' in tag:
        tag = tag.split('-')[0]
    # -NONE- tag
    if "" == tag:
        return " "
    return tag


Now after we did so we can use this function to create all productions given a tree. 

In [ ]:
def get_tag(tree):
    if isinstance(tree, Tree):
        simplified_tag = simplify_functional_tag(tree.label())
        # -NONE- tag
        if simplified_tag == " ":
            return " "
        return Nonterminal(simplify_functional_tag(tree.label()))
    else:
        return tree

def tree_to_production(tree):
    # if the root is NONE we don't need to parse it more
    if(" " == get_tag(tree)):
        return
    else:
        return Production(get_tag(tree), [get_tag(child) for child in tree])

def tree_to_productions(tree):
    yield tree_to_production(tree)
    for child in tree:
        if isinstance(child, Tree):
            for prod in tree_to_productions(child):
                if prod:
                    yield prod

Once we get all the productions of a given tree, all we have to do is to induce a PCFG from those productions, to so do we will use NLTK function called **induce_pcfg**, what this function does is-

For each production **A -> B C** in a list of productions the probability of it in a PCFG is
<br>$P(B, C|A) = \frac{count(A -> B C)}{count(A -> *)}$ so it's the relation between how many times we've seen *A -> B C* compared to all the possible rules for *A*.

So implementing it in python we get:

In [ ]:
def pcfg_learn(treebank, n):
    trees = treebank.parsed_sents()[:n]
    trees_productions = []
    for tree in trees:
        productions = tree_to_productions(tree)
        trees_productions += [prod for prod in productions]
    pcfg_induced = induce_pcfg(Nonterminal('S'), trees_productions)
    return pcfg_induced


In [ ]:
treebank = LazyCorpusLoader('treebank/combined', BracketParseCorpusReader, r'wsj_.*\.mrg')

#### Part 2.2.1

The first thing we'd like to do is to examine the tree bank a little bit, we'd like to see how many internal nodes are in it, and how many productions are in it (we shall do so for 200 trees and not the entire treebank)

In [ ]:
def count_productions(treebank, n):
    trees = treebank.parsed_sents()[:n]
    trees_productions = []
    for tree in trees:
        productions = tree_to_productions(tree)
        trees_productions += [prod for prod in productions]
    return len([prod for prod in trees_productions])

In [ ]:
def count_tree_nodes(tree):
    if isinstance(tree, Tree):
        child_nodes = 0.0
        for child in tree:
            child_nodes += count_tree_nodes(child)
        return(1 + child_nodes)
    else:
        # leave
        return 0

def count_internal_nodes(treebank, n):
    trees = treebank.parsed_sents()[:n]
    trees_internals = 0.0
    for tree in trees:
        trees_internals += count_tree_nodes(tree)
    return trees_internals

In [ ]:
print("Number of productions:", count_productions(treebank, 200))
print("Number of internals:", count_internal_nodes(treebank, 200))

As we can see, there's a slight difference, that might be due to the fact we haven't removed the NONE tags from the internal node counts, so lets see how many NONE tags are in there:

In [ ]:
def count_nones_in_tree(tree):
    if isinstance(tree, Tree):
        tree_tag = tree.label()
        child_nodes = 0.0
        if("NONE" in tree_tag):
            child_nodes = 1        
        for child in tree:
            child_nodes += count_nones_in_tree(child)
        return(child_nodes)
    else:
        # leave
        return 0

def count_none(treebank, n):
    trees = treebank.parsed_sents()[:n]
    none_trees_count = 0.0
    for tree in trees:
        none_trees_count += count_nones_in_tree(tree)
    return none_trees_count

In [ ]:
print("Number of nones:", count_none(treebank, 200))

And as expected all the "missing" productions are for the "NONE" elements. 
<br> this result fits what we learned in class - the number of internal nodes is equal to the number of productions

#### Part 2.2.2

Instead of doing what we did in the previous section , we don't want to build a pcfg now, we just to look at the frequencies, so what we will do is build a freqDist

In [ ]:
def pcfg_freq(treebank, n):
    trees = treebank.parsed_sents()[:n]
    trees_productions = []
    for tree in trees:
        productions = tree_to_productions(tree)
        trees_productions += [prod for prod in productions]
    pcfg_freq = Counter(trees_productions)
    return pcfg_freq

We want to examine the frequencies on 200 trees from the treebank:

In [ ]:
two_hundred_trees_freq = pcfg_freq(treebank, 200)

Lets plot the distribution of productions according to their frequency:

In [ ]:
def plot_distributions(tree_freq, fignum):
    _, counts = zip(*tree_freq.items())
    counts = (list(counts))
    counts.sort()

    labels, values = zip(*Counter(counts).items())
    indexes = np.arange(len(labels))


    plt.figure(num=fignum, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
    plt.bar(indexes, values, 1.5)
    plt.xticks(indexes, labels)
    plt.show()

In [ ]:
plot_distributions(two_hundred_trees_freq, 3)

As we can see, most productions appear only once, it is very rare for a production to appear over 10 times for example. This makes sense considering Penn Treebank is based on paper articles which has a very diverse English, so the productions would be unique.

#### Part 2.2.3

In [ ]:
two_hundred_trees_pcfg = pcfg_learn(treebank, 200)
four_hunred_trees_pcfg = pcfg_learn(treebank, 400)

#TODO - compare

### Part 2.3

Now instead of calculating the PCFG of a simple treebank, we'd like to calculate it for a CNF treebank.

The firat thing we need to do before transforming a tree into a CNF tree, is to simplify the tree - meaning get rid of NONE tags and simplify the tags.
<br>We treat a NONE brunch as a brunch we can remove from the tree because it leads no where.
<br>The code below simplifies a tree:

In [2]:
def simplify_functional_tag(tag):
    if '-' in tag:
        tag = tag.split('-')[0]
    # -NONE- tag
    if "" == tag:
        return " "
    return tag

def get_tag_clean(tree):
    if isinstance(tree, Tree):
        simplified_tag = simplify_functional_tag(tree.label())
        return simplified_tag

def build_simplified_tree(tree):
    if isinstance(tree, Tree):
        root_tag = get_tag_clean(tree)
        if(" " == root_tag):
            return " "
        else:
            simplified_child_nodes = [build_simplified_tree(child) for child in tree]
            # remove all NONE children
            simplified_children = [child for child in  simplified_child_nodes if child != " "]
            if(len(simplified_children) >= 1):
                return(Tree(root_tag, simplified_children))
            else:
                # This means the only child is NONE so we can remove it from it's parent as well
                return " "
    else:
        return tree

After we do so we just have to turn a tree into production (notice we don't have to handle the NONE cases anymore as our tree is clear of them).

In [3]:
def get_tag_or_value(tree):
    if isinstance(tree, Tree):
        return Nonterminal(tree.label())
    else:
        return tree
        
def cnf_tree_to_production(tree):
    return Production(get_tag_or_value(tree),[get_tag_or_value(child) for child in tree])

def cnf_tree_to_productions(tree):
    yield cnf_tree_to_production(tree)
    for child in tree:
        if isinstance(child, Tree):
            for prod in cnf_tree_to_productions(child):
                if prod:
                    yield prod

Now after setting up the functions to all stages, letse set it all together to induce a PCFG from a CFG treebank - 
The stages are, for each tree in the treebank:
- Simplify the tree and rid of NONE tags.
- Transform the tree into a CNF tree.
- Induce the productions from the CNF tree.

In [4]:
def pcfg_cnf_learn(treebank, n):
    trees = treebank.parsed_sents()[:n]
    trees_productions = []
    for tree in trees:
        new_tree = build_simplified_tree(tree)
        chomsky_normal_form(new_tree, factor='right', horzMarkov=1, vertMarkov=1, childChar='|', parentChar='^')
        productions = cnf_tree_to_productions(new_tree)
        trees_productions += [prod for prod in productions]
    pcfg_induced = induce_pcfg(Nonterminal('S'), trees_productions)
    return pcfg_induced

In [ ]:
treebank = LazyCorpusLoader('treebank/combined', BracketParseCorpusReader, r'wsj_.*\.mrg')
pcfg_cnf = two_hundred_trees_pcfg = pcfg_cnf_learn(treebank, 200)

We want to show the number of productions in the CNF treebank - so we get:

In [ ]:
print("The number of productions:", len(pcfg_cnf.productions()))

We've seen earlier that in the original treebank we had 8692 internal nodes, now we have 2797 productions, but these are unique productions, to find out exactly how many productions we have (not only unique ones) lets count the amount of internal nodes in the CNF treebank:

In [ ]:
def pcfg_cnf_internals(treebank, n):
    trees = treebank.parsed_sents()[:n]
    trees_internals = 0.0
    for tree in trees:
        new_tree = build_simplified_tree(tree)
        chomsky_normal_form(new_tree, factor='right', horzMarkov=1, vertMarkov=1, childChar='|', parentChar='^')
        trees_internals += count_tree_nodes(new_tree)
    return trees_internals

In [ ]:
print("Internal nodes/ number of productions:", pcfg_cnf_internals(treebank, 200))

So we can see turning it into a CNF added more productions(and internal nodes) over all. This makes sense considering a CNF tree is usally bigger than the non-CNF tree.

#TODO - add conclusions

### Part 2.4

What we'd like to do now is to explore the CFG hypothesis that a node expansion is independent from its location within a tree.
<br>We will do so by exploring the expansion of the NP category in thr CNF trees.

The first thing we'd like to do is show how the NP's RHS and LHS distributed , meaning for each LHS show the distribution of it's RHS. to do so we will look at each NP tag in the treebank and see it's production.

In [ ]:
def get_np_productions(treebank, n):
    trees = treebank.parsed_sents()[:n]
    np_productions = []
    for tree in trees:
        productions = tree_to_productions(tree)
        for prod in productions:
            if(prod.lhs() == Nonterminal('NP')):
                np_productions += [(prod.rhs())]
    return np_productions

In [ ]:
np_productions = get_np_productions(treebank, 200)

Now that we have the productions we's like to look at an histogram of them, to do so we will use the counter object.

In [ ]:
counter = Counter(np_productions)
counts = counter.most_common()

ind = [count for key, count in counts]
bins = range(0, len(counts))
bins = [bin * 5 for bin in bins]

keys = [key for key, count in counts]

plt.figure(num=6, figsize=(18, 10), dpi=80, facecolor='w', edgecolor='k')
plt.bar(bins, ind , align='edge', width=1)
plt.xticks(bins, keys, rotation='vertical')
plt.show()

Now we'd like to to see what is the distribution of NP that comes after S, to so we need to get all NP productions after an S:

In [ ]:
def tree_to_np_under_x_productions(x, tree, is_child):
    productions = []
    if isinstance(tree, Tree):
        if(get_tag(tree) == Nonterminal(x)):
            for child in tree:
                productions += tree_to_np_under_x_productions(x, child, True)
            return productions
        elif(get_tag(tree) == Nonterminal('NP') and is_child):
            productions += [Production(get_tag(tree), [get_tag(child) for child in tree])]  
        for child in tree:
            productions += tree_to_np_under_x_productions(x, child, False) 
    return productions
    
    
def get_np_s_production(treebank, n):
    trees = treebank.parsed_sents()[:n]
    np_s_productions = []
    for tree in trees:
        np_s_productions += tree_to_np_under_x_productions('S', tree, False)
    np_s_productions = [prod.rhs() for prod in np_s_productions]
    return np_s_productions

In [ ]:
np_s_productions = get_np_s_production(treebank, 200)
counter_s_np = Counter(np_s_productions)
counts_s_np = counter_s_np.most_common()

ind_s_np = [count for key, count in counts_s_np]
bins_s_np = range(0, len(counts_s_np))
bins_s_np = [bin * 10 for bin in bins_s_np]
keys_s_np = [key for key, count in counts_s_np]

plt.figure(num=7, figsize=(20, 10), dpi=80, facecolor='w', edgecolor='k')
plt.bar(bins_s_np, ind_s_np , align='edge', width=1)
plt.xticks(bins_s_np, keys_s_np, rotation='vertical')
plt.show()

And now lets do the same for 'VP' tags:

In [ ]:
def get_np_vp_production(treebank, n):
    trees = treebank.parsed_sents()[:n]
    np_vp_productions = []
    for tree in trees:
        np_vp_productions += tree_to_np_under_x_productions('VP', tree, False)
    np_vp_productions = [prod.rhs() for prod in np_vp_productions]
    return np_vp_productions

In [ ]:
np_vp_productions = get_np_vp_production(treebank, 200)
counter_vp_np = Counter(np_vp_productions)
counts_vp_np = counter_vp_np.most_common()

ind_vp_np = [count for key, count in counts_vp_np]
bins_vp_np = range(0, len(counts_vp_np))
bins_vp_np = [bin * 10 for bin in bins_vp_np]
keys_vp_np = [key for key, count in counts_vp_np]

plt.figure(num=7, figsize=(20, 10), dpi=80, facecolor='w', edgecolor='k')
plt.bar(bins_s_np, ind_s_np , align='edge', width=1)
plt.xticks(bins_vp_np, keys_vp_np, rotation='vertical')
plt.show()

Now what we'd like to do is to compare those distributions, so lets first turn all of the frequency plots into proper distributions:

In [ ]:
np_freq = nltk.ConditionalFreqDist((Nonterminal('NP'), prod) for prod in np_productions)
np_s_freq = nltk.ConditionalFreqDist((Nonterminal('NP'), prob) for prob in np_s_productions)
np_vp_freq = nltk.ConditionalFreqDist((Nonterminal('NP'), prob) for prob in np_vp_productions)

np_probs = nltk.ConditionalProbDist(np_freq, nltk.MLEProbDist)
np_s_probs = nltk.ConditionalProbDist(np_s_freq, nltk.MLEProbDist)
np_vp_probs = nltk.ConditionalProbDist(np_vp_freq, nltk.MLEProbDist)

And now we'd like to compare each pair using KL-Divergence we used earlier.

In [ ]:
def compute_divergence_np(prob_a, prob_b):
    divergence_np = {}
    for condition in prob_a.conditions():
        kl_np_s = compute_kl_divergence(prob_a[condition], prob_b[condition])
        divergence_np[condition] = kl
    return(divergence_np)

Lets start by comparing the entire treebank to the ones starting with s:

In [ ]:
compute_divergence_np(np_probs, np_s_probs)

In [ ]:
compute_divergence_np(np_probs, np_vp_probs)

In [ ]:
compute_divergence_np(np_s_probs, np_vp_probs)

#TODO - conclusions

## Part 3

In this part of the assignment we will construct a viterbi parser for the previously induced PCFG and then evaluate it.

### Part 3.1 - building the parser

#### Part 3.1.1

Before we go ahead and build a parser, we need to set up our training data and our test data, to do so we will split the Penn Treebank into two parts - the training corpus will contain 80% of the trees, and the testing corpus will contain the other 20%.

Before we do so, lets explore the treebank so we know the length each set should be:

In [ ]:
treebank = LazyCorpusLoader('treebank/combined', BracketParseCorpusReader, r'wsj_.*\.mrg')
trees = treebank.parsed_sents()
total = len(trees)
eighty_pr = int(total * 0.8)
twenty_pr = int(total * 0.2)
print("80%: ", eighty_pr)
print("20%: ", twenty_pr)

As we can see, the split fits the division from the question - about 3200 train trees and about 800 test trees.

So now we know the exact size each should have, and we can write functions to get each:

In [5]:
def get_data(divider, end):
    treebank = LazyCorpusLoader('treebank/combined', BracketParseCorpusReader, r'wsj_.*\.mrg')
    trees_start = treebank.parsed_sents()[:divider]
    trees_end = treebank.parsed_sents()[divider : end - 1]
    return trees_start, trees_end

In [6]:
train_set, test_set = get_data(3131, 3131 + 782)

#### Part 3.1.2

The next thing we'd like to do is to learn a PCFG over the Chomsky Normal Form version of the treebank. 
To do so we will use the functions from part 2 and the training set size as the *n* input.

In [7]:
treebank = LazyCorpusLoader('treebank/combined', BracketParseCorpusReader, r'wsj_.*\.mrg')
pcfg_training = pcfg_cnf_learn(treebank, 3131)

#### Part 3.1.3

Now once we have a PCFG we can finally construct a **Viterbi Parser** for our treebank. 
To do so we will use NLTK native function to build such parser, and we will feed it with our PCFG.

In [8]:
v_parser = nltk.ViterbiParser(pcfg_training)

Now lets test our parser on a few example sentences:

In [ ]:
sent = 'any country is fine , as long as you know how to drive'.split()
p = v_parser.parse(sent)
for t in p:
    print(t)

And another few:

In [ ]:
sent = 'we saw a big hill and we climbed on top of it'.split()
p = v_parser.parse(sent)
for t in p:
    print(t)

In [ ]:
sent = "it does not take much to be happy , you just need to try".split()
p = v_parser.parse(sent)
for t in p:
    print(t)

As we can see, the sentences I gave are not that likely - this makes sense considering this treebank is composed out of journalisem articles, just for fun lets try it over a more journalistic sentence:

In [ ]:
sent = "many banks are turning away from strict price competition".split()
p = v_parser.parse(sent)
for t in p:
    print(t)

And as we can see it is actually more likely than the others.

Something to note is that this parser can't deal with unknown words, which is not all that great:

In [ ]:
sent = 'big cat'.split()
p = v_parser.parse(sent)
try:
    for t in p:
        print(t)
except ValueError as e:
    print(e)

This might be a problem if our test set and train set for example don't have the same exact vocabulary, so we need to check that out:

In [9]:
training_vocab = set({})
for tree in train_set:
    words = tree.leaves()
    for word in words:
        training_vocab.add(word)
    
test_vocab = set({})
for tree in test_set:
    words = tree.leaves()
    for word in words:
        test_vocab.add(word)

In [ ]:
print("training_vocab", len(training_vocab))
print("testing_vocab", len(test_vocab))

So we can clearly see there are a lot of words in the training that are not in the test dataset(this will also be usefull later), but what about the opposite?
<br>Lets look at the intersection of the sets: 

In [ ]:
print(len(training_vocab.intersection(test_vocab)))

so we have $4071-2712 = 1359$ words that are in the test set, but not in the train set.
<br>What will we do about it? we have to add grammer rules to deal with such words - to do so we will use their POS tag(like we learned in class).

In [10]:
in_test = (test_vocab - training_vocab)
print(len(in_test))

1364


First thing we will do is to convert the words to rules based on the POS tagging. A thing to note about that is that nltk pos tagger uses the univeral tagset, but our parser uses ptb tagset, so we need to convert the tags after figuring out the tag for our word, so lets start building up our rules(we will later construct them into productions):

In [11]:
def get_ptb_tag(word):
    return nltk.pos_tag([word])

In [12]:
pos_tags = []
for word in in_test:
    pos_tags += get_ptb_tag(word)

Now we need to construct productions from those rules:

In [13]:
pos_productions = []
for tag in pos_tags:
    pos_productions += [Production(Nonterminal(tag[1]), [tag[0]])]

So now we have a list of all the prductions for the missing words, and we need to somehow insert them into our grammer.

To do so we will modify out pcfg generator function:

In [125]:
def pcfg_cnf_learn_with_productions(treebank, n, additional_productions):
    trees = treebank.parsed_sents()[:n]
    trees_productions = []
    for tree in trees:
        new_tree = build_simplified_tree(tree)
        chomsky_normal_form(new_tree, factor='right', horzMarkov=1, vertMarkov=1, childChar='|', parentChar='^')
        productions = cnf_tree_to_productions(new_tree)
        trees_productions += [prod for prod in productions]
    trees_productions += additional_productions
    pcfg_induced = induce_pcfg(Nonterminal('S'), trees_productions)
    return pcfg_induced

And now we will construct a viterbi parser with the additional POS productions:

In [126]:
treebank = LazyCorpusLoader('treebank/combined', BracketParseCorpusReader, r'wsj_.*\.mrg')
pcfg_pos_training = pcfg_cnf_learn_with_productions(treebank, 3131, pos_productions)
v_pos_parser = nltk.ViterbiParser(pcfg_pos_training)

Now lets test our parser on a word in the test trees which isn't in the train trees - to do so we will find such word:

In [ ]:
word = in_test.pop()
print(word)

and as we can see the word *richer* is in the test set but not in the train set, if we use the old parser we defined we will get an error:

In [ ]:
sent = 'you are richer'.split()
p = v_parser.parse(sent)
try:
    for t in p:
        print(t)
except ValueError as e:
    print(e)

But using our new parser we get:

In [ ]:
sent = 'you are richer'.split()
p = v_pos_parser.parse(sent)
try:
    for t in p:
        print(t)
except ValueError as e:
    print(e)

And as we can see, we managed to parse the tree.

### Part 3.2

After writing the Viterbi parser, the next thing we'd like to do is to determine how well it preforms, to do so we will use the trusty old measure we defined in the previous assignment - Precision, Recall and Fscore, but a modified version of them.

What we'd like to is to evaluate them in two methods:
- unlabeled: comparing first and last index, regardless of the label.
- labeled: comparing both the label and the indices.

The mentioned index refers to the index which is part of the constituents. 
<br>A constituent is a triplet of the form *(interior node, index of the first word in the sentence covered by it, index of the last word in the sentence covered by it)*.

**Labeled Precision** is therfore defined as the average of how many brackets
in the resulting parse tree match those in the actual tree. we measure it by: 
$LP = \frac{Correct Constituents Parse S}{Total Constituents Parse S}$

**Labeled Recall** is the average of how many brackets in the actual tree are in the resulting parse tree, and we measure it by:
$LR = \frac{Correct Constituents Parse S}{Total Constituents Reference Parse S}$

The F-Measure stays same as before - an harmonic average of precision and recall.

The first thing we need to do then is given a tree - we want to build it's constituents:

In [82]:
def flatten(l): return flatten(l[0]) + (flatten(l[1:]) if len(l) > 1 else []) if type(l) is list else [l]

def parse_inner_labels(tree, final_tags, prev_tags, list_of_consts):
    if(isinstance(tree, Tree)):
        if(tree.label() in final_tags):
            return prev_tags
        else:
            child_indices = []
            p_t = prev_tags
            for child in tree:
                child_indices.append([parse_inner_labels(child, final_tags, p_t, list_of_consts)])
                p_t += len(child.pos())
            child_indices = flatten(child_indices)
            list_of_consts.append((tree.label(), child_indices[0], child_indices[len(child_indices) - 1] + 1))
            return child_indices

def build_constituents(tree):
    tagged_words = tree.pos()
    final_tags = [b for (a, b) in tagged_words]
    leaves = tree.leaves()
    list_or_consts = []
    parse_inner_labels(tree, final_tags, 0, list_or_consts)
    return list_or_consts

And we will test our function using the examples in the assignment on parsing evaluation by *Scott Farrar*:

In [83]:
parse_tree = Tree(Nonterminal('S') ,[Tree(Nonterminal('NP'),[Tree(Nonterminal('A'), ["a"])]), Tree(Nonterminal('VP'), [Tree(Nonterminal('B') ,["b"]), Tree(Nonterminal('PP'), [Tree(Nonterminal('C'), ["c"])])])])
consts = build_constituents(parse_tree)
print(consts)
second_parse_tree = Tree(Nonterminal('S'), [Tree(Nonterminal('NP'), [Tree(Nonterminal('A'), ["a"])]), Tree(Nonterminal('VP'), [Tree(Nonterminal('B') ,["b"])] ), Tree(Nonterminal('PP') ,[Tree(Nonterminal('C'), ["c"])])])
consts_two = build_constituents(second_parse_tree)
print(consts_two)

[(NP, 0, 1), (PP, 2, 3), (VP, 1, 3), (S, 0, 3)]
[(NP, 0, 1), (VP, 1, 2), (PP, 2, 3), (S, 0, 3)]


We should also test it with a bit more challenging trees, for example one that has the same word in few different locations(and therfore under different rules), for example if we take the tree

In [84]:
parse_tree = Tree(Nonterminal('S'), [Tree(Nonterminal('CC'), ['But']), Tree(Nonterminal('NP'), [Tree(Nonterminal('NN'), ['revenue'])]), Tree(Nonterminal('VP'), [Tree(Nonterminal('VBD'), ['declined']), Tree(Nonterminal('PP'), [Tree(Nonterminal('TO') ,['to']), Tree(Nonterminal('NP'), [Tree(Nonterminal('QP'), [Tree(Nonterminal('$'), ['$']), Tree(Nonterminal('CD'), ['3']), Tree(Nonterminal('CD'), ['billion'])])])]), Tree(Nonterminal('PP'), [Tree(Nonterminal('IN'), ['from']), Tree(Nonterminal('NP'), [Tree(Nonterminal('QP'), [Tree(Nonterminal('$'), ['$']), Tree(Nonterminal('CD'), ['3.2']), Tree(Nonterminal('CD'), ['billion'])])])])]), Tree(Nonterminal('.'), ['.'])])
parse_tree.pprint()
print(parse_tree.leaves())

(S
  (CC But)
  (NP (NN revenue))
  (VP
    (VBD declined)
    (PP (TO to) (NP (QP ($ $) (CD 3) (CD billion))))
    (PP (IN from) (NP (QP ($ $) (CD 3.2) (CD billion)))))
  (. .))
['But', 'revenue', 'declined', 'to', '$', '3', 'billion', 'from', '$', '3.2', 'billion', '.']


We can see "$" shows up two times in different index - one time at 4 and one time at 8, so we should be able to see that in our constituents as well, meaning we should see 'QP' derived to (4, 7) once and to (8, 11) once.

In [85]:
consts = build_constituents(parse_tree)
print(consts)

[(NP, 1, 2), (QP, 4, 7), (NP, 4, 7), (PP, 3, 7), (QP, 8, 11), (NP, 8, 11), (PP, 7, 11), (VP, 2, 11), (S, 0, 12)]


And as we can see it does handle both cases differently.

The next thing we'll do is write functions which calculates both labeled and unlabeled precision and recall:

In [19]:
def calc_labeled_precision(parse_tree_constituents, actual_tree_constituents):
    total_constituents = len(parse_tree_constituents)
    total_correct = 0
    for constituent in parse_tree_constituents:    
        if(constituent in actual_tree_constituents):
            total_correct += 1
    return(total_correct / float(total_constituents))

In [20]:
def calc_labeled_recall(parse_tree_constituents, actual_tree_constituents):
    total_constituents_gold = len(actual_tree_constituents)
    total_correct = 0.0
    for constituent in parse_tree_constituents:    
        if(constituent in actual_tree_constituents):
            total_correct += 1
    return(total_correct / float(total_constituents_gold))    

And for the unlabeled version we need to check just if the indices match:

In [21]:
def calc_unlabeled_precision(parse_tree_constituents, actual_tree_constituents):
    total_constituents = len(parse_tree_constituents)
    total_correct = 0
    for label, s_i, e_i in parse_tree_constituents:
        for a_label, a_s_i, a_e_i in actual_tree_constituents:
            if(a_s_i == s_i and a_e_i == e_i):
                total_correct += 1
                break
    return(total_correct / float(total_constituents))

In [86]:
def calc_unlabeled_recall(parse_tree_constituents, actual_tree_constituents):
    total_constituents_gold = len(actual_tree_constituents)
    total_correct = 0.0
    for label, s_i, e_i in parse_tree_constituents:
        for a_label, a_s_i, a_e_i in actual_tree_constituents:
            if(a_s_i == s_i and a_e_i == e_i):
                total_correct += 1
                break
    return(total_correct / float(total_constituents_gold))

And after defining the measures all we have to do is use them in order to evaluate our parser on the test dataset - we'll try it first with just the first tree:

In [127]:
def parse_first_tree(test_set, parser):
    for tree in test_set:
        simplified = build_simplified_tree(tree)
        sentence = simplified.leaves()
        chomsky_normal_form(simplified, factor='right', horzMarkov=1, vertMarkov=1, childChar='|', parentChar='^')
        actual_tree_constituents = build_constituents(simplified)
        viterbi_parsed = parser.parse(sentence)
        for t in viterbi_parsed:
            parsed_tree_constituents = build_constituents(t)
            print("labeld precision", calc_labeled_precision(parsed_tree_constituents, actual_tree_constituents))
            print("labeled recall", calc_labeled_recall(parsed_tree_constituents, actual_tree_constituents))
            print("unlabeled precision", calc_unlabeled_precision(parsed_tree_constituents, actual_tree_constituents))
            print("unlabeled recall", calc_unlabeled_recall(parsed_tree_constituents, actual_tree_constituents))
        break

In [ ]:
parse_first_tree(test_set, v_pos_parser)

This code does work, but running it takes about 10 minutes which is really bad considering we want to run this over the entire treebank.
<br>There are a few reasons for that
- As we learned in class the parsing time takes $O(n^{3}|R|)$ where *R* is the set of rules.
- We're running our code on CFG trees, which in general have more rules than the original trees, so we increased $|R|$ by a lot.
- As we've previously seen there are much more words in th training set than in the test set, so there are a lot of unuseful rules.

In order to solve this problem we need to either have a smaller $n$ or a smaller $R$, and considering we don't want to cut off parts of our sentences, we'd go for the second option.

When going through the literature on how to do so we found a few possible ways:
- *Beam threshholding*: Assume low probability partial parses unlikely to yield high probability overall, Keep only top k most probably partial parses.
- *Heuristic Filtering*: Some rules/partial parses are unlikely to end up in best parse. Don’t store those rules. use a threshold to remove low probability rules.

The second option sounded more do-able to us, we'll removelow frequency rules before we construct a pcfg in order to do so, but to do that we first need to examine the rules to see what's a good threshold is:

In [94]:
def pcfg_cnf_examine_productions(treebank, n):
    trees = treebank.parsed_sents()[:n]
    trees_productions = []
    for tree in trees:
        new_tree = build_simplified_tree(tree)
        chomsky_normal_form(new_tree, factor='right', horzMarkov=1, vertMarkov=1, childChar='|', parentChar='^')
        productions = cnf_tree_to_productions(new_tree)
        trees_productions += [prod for prod in productions]
    c = Counter(trees_productions)
    return c

In [95]:
treebank = LazyCorpusLoader('treebank/combined', BracketParseCorpusReader, r'wsj_.*\.mrg')
counter_pcfg = pcfg_cnf_examine_productions(treebank, 3131)

length: 158985


So now we can explore the rules a little bit:

In [99]:
for c in counter_pcfg:
    print("Rule: {}, Count: {}".format(c, counter_pcfg[c]))

Rule: S -> NP^<S> S|<VP>, Count: 1410
Rule: NP^<S> -> NP^<NP> NP|<,>^<S>, Count: 270
Rule: NP^<NP> -> NNP NNP, Count: 436
Rule: NNP -> 'Pierre', Count: 1
Rule: NNP -> 'Vinken', Count: 2
Rule: NP|<,>^<S> -> , NP|<ADJP>^<S>, Count: 7
Rule: , -> ',', Count: 3955
Rule: NP|<ADJP>^<S> -> ADJP^<NP> ,, Count: 7
Rule: ADJP^<NP> -> NP^<ADJP> JJ, Count: 7
Rule: NP^<ADJP> -> CD NNS, Count: 7
Rule: CD -> '61', Count: 3
Rule: NNS -> 'years', Count: 103
Rule: JJ -> 'old', Count: 20
Rule: S|<VP> -> VP^<S> ., Count: 2414
Rule: VP^<S> -> MD VP^<VP>, Count: 569
Rule: MD -> 'will', Count: 216
Rule: VP^<VP> -> VB VP|<NP>^<VP>, Count: 356
Rule: VB -> 'join', Count: 4
Rule: VP|<NP>^<VP> -> NP^<VP> VP|<PP>^<VP>, Count: 73
Rule: NP^<VP> -> DT NN, Count: 281
Rule: DT -> 'the', Count: 3268
Rule: NN -> 'board', Count: 22
Rule: VP|<PP>^<VP> -> PP^<VP> NP^<VP>, Count: 16
Rule: PP^<VP> -> IN NP^<PP>, Count: 2139
Rule: IN -> 'as', Count: 271
Rule: NP^<PP> -> DT NP|<JJ>^<PP>, Count: 323
Rule: DT -> 'a', Count: 1449
Ru

Rule: VP|<,>^<S> -> , VP|<``>^<S>, Count: 20
Rule: VP|<``>^<S> -> `` S^<VP>, Count: 24
Rule: `` -> '``', Count: 622
Rule: S^<VP> -> NP^<S> VP^<S>, Count: 148
Rule: NP^<S> -> DT, Count: 60
Rule: DT -> 'This', Count: 33
Rule: NP^<VP> -> DT NP|<JJ>^<VP>, Count: 176
Rule: DT -> 'an', Count: 246
Rule: NP|<JJ>^<VP> -> JJ NN, Count: 167
Rule: NN -> 'story', Count: 5
Rule: PRP -> 'We', Count: 44
Rule: VP^<S> -> VBP VP^<VP>, Count: 276
Rule: VBP -> "'re", Count: 27
Rule: VP^<VP> -> VBG PP^<VP>, Count: 42
Rule: VBG -> 'talking', Count: 4
Rule: PP^<VP> -> IN ADVP^<PP>, Count: 5
Rule: IN -> 'about', Count: 132
Rule: ADVP^<PP> -> ADVP^<ADVP> SBAR^<ADVP>, Count: 1
Rule: ADVP^<ADVP> -> NP^<ADVP> IN, Count: 1
Rule: SBAR^<ADVP> -> IN S^<SBAR>, Count: 8
Rule: IN -> 'before', Count: 35
Rule: NP^<S> -> NN, Count: 90
Rule: NN -> 'anyone', Count: 6
Rule: VP^<S> -> VBD PP^<VP>, Count: 99
Rule: VBD -> 'heard', Count: 1
Rule: VBG -> 'having', Count: 10
Rule: DT -> 'any', Count: 91
Rule: NP|<JJ>^<VP> -> JJ NNS,

Rule: VBP -> 'expect', Count: 6
Rule: JJ -> 'further', Count: 6
Rule: NNS -> 'declines', Count: 4
Rule: NN -> 'interest', Count: 41
Rule: NNS -> 'rates', Count: 44
Rule: NP|<JJ>^<NP> -> JJ NP|<JJ>^<NP>, Count: 51
Rule: JJ -> 'average', Count: 20
Rule: NP|<JJ>^<NP> -> JJ NP|<NN>^<NP>, Count: 114
Rule: JJ -> 'seven-day', Count: 3
Rule: NN -> 'compound', Count: 4
Rule: NN -> 'yield', Count: 19
Rule: NP^<NP> -> DT NP|<CD>^<NP>, Count: 36
Rule: NP|<CD>^<NP> -> CD NP|<JJ>^<NP>, Count: 8
Rule: CD -> '400', Count: 5
Rule: JJ -> 'taxable', Count: 2
Rule: VBN -> 'tracked', Count: 1
Rule: NNP -> 'IBC', Count: 1
Rule: NNP -> 'Money', Count: 3
Rule: NNP -> 'Fund', Count: 12
Rule: NNP -> 'Report', Count: 8
Rule: VBD -> 'eased', Count: 2
Rule: NN -> 'fraction', Count: 1
Rule: NP^<PP> -> DT NP|<NN>^<PP>, Count: 158
Rule: NN -> 'point', Count: 24
Rule: VP|<PP>^<S> -> PP^<VP> VP|<PP>^<S>, Count: 34
Rule: NP^<PP> -> CD NN, Count: 55
Rule: CD -> '8.45', Count: 2
Rule: NN -> '%', Count: 328
Rule: VP|<PP>^<

Rule: RB -> 'effectively', Count: 4
Rule: VP|<ADJP>^<VP> -> ADJP^<VP> PP^<VP>, Count: 10
Rule: ADJP^<VP> -> NP^<ADJP> JJR, Count: 1
Rule: NP^<ADJP> -> CD NN, Count: 3
Rule: CD -> '7.5', Count: 2
Rule: IN -> 'per', Count: 5
Rule: NN -> 'subscriber', Count: 1
Rule: VBZ -> 'costs', Count: 4
Rule: NP^<VP> -> QP^<NP>, Count: 29
Rule: QP|<$>^<NP> -> $ CD, Count: 18
Rule: CD -> '120,000', Count: 1
Rule: VP^<S> -> VBZ VP|<ADVP>^<S>, Count: 74
Rule: VP|<ADVP>^<S> -> ADVP^<VP> S^<VP>, Count: 8
Rule: RB -> 'yet', Count: 9
Rule: VB -> 'announce', Count: 2
Rule: NP^<VP> -> PRP$ NP|<CD>^<VP>, Count: 5
Rule: NP|<CD>^<VP> -> CD NP|<NN>^<VP>, Count: 3
Rule: NNP -> 'Circulation', Count: 1
Rule: NNP -> 'Credit', Count: 2
Rule: NNP -> 'Plan', Count: 2
Rule: VBZ -> 'awards', Count: 1
Rule: NP^<VP> -> NP^<NP>, Count: 28
Rule: NNS -> 'credits', Count: 1
Rule: PP|<``>^<VP> -> `` NP^<PP>, Count: 8
Rule: NN -> 'renewal', Count: 1
Rule: VB -> 'reward', Count: 2
Rule: VP|<PP>^<VP> -> PP^<VP> VP|<NP>^<VP>, Count: 

Rule: CD -> '20.5', Count: 1
Rule: NN -> 'loss', Count: 15
Rule: VBN -> 'reached', Count: 12
Rule: NN -> 'comment', Count: 10
Rule: VP^<VP> -> VP^<VP> PRN^<VP>, Count: 1
Rule: VB -> 'work', Count: 7
Rule: NP|<JJ>^<PP> -> JJ NP|<NAC>^<PP>, Count: 1
Rule: NP|<NAC>^<PP> -> NAC^<NP> NN, Count: 3
Rule: NAC^<NP> -> NNP NAC|<NNPS>^<NP>, Count: 1
Rule: NNP -> 'Colorado', Count: 2
Rule: NAC|<NNPS>^<NP> -> NNPS NAC|<,>^<NP>, Count: 1
Rule: NNPS -> 'Springs', Count: 1
Rule: NNP -> 'Colo.', Count: 5
Rule: NN -> 'contractor', Count: 2
Rule: NP^<PRN> -> NP^<NP> NP|<SBAR>^<PRN>, Count: 1
Rule: NN -> 'arrangement', Count: 3
Rule: NP|<SBAR>^<PRN> -> SBAR^<NP> PP^<NP>, Count: 1
Rule: VBN -> 'Regarded', Count: 1
Rule: NN -> 'father', Count: 4
Rule: VBN -> 'paid', Count: 13
Rule: CD -> '600,000', Count: 2
Rule: IN -> 'At', Count: 26
Rule: CD -> '240,000', Count: 1
Rule: NNP -> 'Messrs.', Count: 1
Rule: VBZ -> 'includes', Count: 7
Rule: NP^<VP> -> NP^<NP> NP|<:>^<VP>, Count: 20
Rule: NNP -> 'Neil', Count: 

...

Rule: NNP -> 'Troubled', Count: 1
Rule: NNP -> 'NBI', Count: 5
Rule: NP^<VP> -> QP^<NP> NP|<PRP$>^<VP>, Count: 1
Rule: QP|<IN>^<NP> -> IN PDT, Count: 1
Rule: NP|<PRP$>^<VP> -> PRP$ NP|<NN>^<VP>, Count: 1
Rule: VP|<VP>^<S> -> VP^<VP> S^<VP>, Count: 1
Rule: VBG -> 'discontinuing', Count: 1
Rule: VB -> 'focus', Count: 3
Rule: NNS -> 'quarters', Count: 1
Rule: RB -> 'any', Count: 2
Rule: VP|<ADVP>^<VP> -> ADVP^<VP> VP|<VB>^<VP>, Count: 1
Rule: RB -> 'Altogether', Count: 1
Rule: CD -> '266', Count: 1
Rule: NNP -> 'Boulder', Count: 2
Rule: CD -> '176', Count: 1
Rule: NP^<PP> -> PRP$ NP|<UCP>^<PP>, Count: 1
Rule: UCP|<CC>^<NP> -> CC UCP|<NNP>^<NP>, Count: 1
Rule: UCP|<NNP>^<NP> -> NNP NNP, Count: 1
Rule: NNP -> 'Kingdom', Count: 3
Rule: VB -> 'fall', Count: 5
Rule: NNP -> 'Jerritts', Count: 2
Rule: VB -> 'commit', Count: 1
Rule: RB -> 'Further', Count: 2
Rule: QP^<NP> -> CC CD, Count: 2
Rule: RB -> 'flat', Count: 1
Rule: VP|<VBD>^<S> -> VBD ADVP^<VP>, Count: 1
Rule: ADVP^<VP> -> IN PP^<ADVP>,

Rule: NP^<PP> -> JJ NP|<UCP>^<PP>, Count: 1
Rule: NP|<UCP>^<PP> -> UCP^<NP> NNS, Count: 1
Rule: UCP^<NP> -> JJ UCP|<,>^<NP>, Count: 1
Rule: JJ -> 'electrical', Count: 3
Rule: UCP|<,>^<NP> -> , UCP|<NN>^<NP>, Count: 3
Rule: UCP|<NN>^<NP> -> NN UCP|<,>^<NP>, Count: 1
Rule: NN -> 'crane-safety', Count: 1
Rule: UCP|<NN>^<NP> -> NN UCP|<CC>^<NP>, Count: 2
Rule: NN -> 'record-keeping', Count: 1
Rule: NN -> 'citation', Count: 1
Rule: VBG -> 'covering', Count: 2
Rule: NP^<VP> -> NP^<NP> NP|<NAC>^<VP>, Count: 1
Rule: NP|<NAC>^<VP> -> NAC^<NP> NP|<NN>^<VP>, Count: 1
Rule: NNP -> 'Clairton', Count: 2
Rule: NN -> 'coke', Count: 1
Rule: NP^<PP> -> JJ NP|<CC>^<PP>, Count: 1
Rule: JJ -> 'electrical-safety', Count: 1
Rule: NNP -> 'Dole', Count: 2
Rule: NNS -> 'citations', Count: 3
Rule: NNS -> 'hazards', Count: 4
Rule: VBD -> 'resulted', Count: 1
Rule: NN -> 'indifference', Count: 1
Rule: JJ -> 'severe', Count: 2
Rule: NNP -> 'Scannell', Count: 2
Rule: NNS -> 'deficiencies', Count: 1
Rule: VP|<``>^<S>

Rule: SBAR^<NP> -> VBN SBAR|<,>^<NP>, Count: 1
Rule: VBN -> 'Provided', Count: 1
Rule: SBAR|<,>^<NP> -> , SBAR|<IN>^<NP>, Count: 1
Rule: SBAR|<IN>^<NP> -> IN S^<SBAR>, Count: 1
Rule: S|<:> -> : S|<.>, Count: 2
Rule: VBZ -> 'signals', Count: 1
Rule: NP|<NN>^<VP> -> NN NP|<PRN>^<VP>, Count: 1
Rule: NP|<PRN>^<VP> -> PRN^<NP> S^<NP>, Count: 1
Rule: NN -> 'pretext', Count: 2
Rule: VBG -> 'guarding', Count: 1
Rule: NN -> 'purse', Count: 2
Rule: NNS -> 'prerogatives', Count: 3
Rule: VBN -> 'specified', Count: 1
Rule: NNP -> 'Article', Count: 10
Rule: NN -> 'species', Count: 1
Rule: VBN -> 'predicated', Count: 1
Rule: NN -> 'interpretation', Count: 4
Rule: NN -> 'clause', Count: 7
Rule: JJ -> 'erroneous', Count: 1
Rule: JJ -> 'unconstitutional', Count: 10
Rule: PP^<VP> -> CC PP^<PP>, Count: 1
Rule: NN -> 'Consequence', Count: 1
Rule: NNS -> 'Appropriations', Count: 1
Rule: NN -> 'Law', Count: 1
Rule: JJ -> 'omnipresent', Count: 1
Rule: JJ -> 'conceivable', Count: 1
Rule: VB -> 'withhold', Coun

Rule: NNP -> 'Jihad', Count: 1
Rule: NN -> 'stock-price', Count: 1
Rule: JJ -> 'band-wagon', Count: 1
Rule: VBZ -> 'picks', Count: 1
Rule: NN -> 'impetus', Count: 1
Rule: VBN -> 'practiced', Count: 1
Rule: NNS -> 'locations', Count: 1
Rule: NNS -> 'widgets', Count: 2
Rule: NN -> 'widget', Count: 7
Rule: RB -> 'constantly', Count: 3
Rule: JJ -> 'index-arbitrage', Count: 4
Rule: VBG -> 'performing', Count: 2
Rule: VB -> 'match', Count: 1
Rule: NP^<S> -> JJ NP|<PRN>^<S>, Count: 1
Rule: JJ -> 'Few', Count: 1
Rule: NP|<PRN>^<S> -> PRN^<NP> NP|<JJ>^<S>, Count: 1
Rule: PRN^<NP> -> , PRN|<IN>^<NP>, Count: 1
Rule: PRN|<IN>^<NP> -> IN PRN|<DT>^<NP>, Count: 1
Rule: PRN|<DT>^<NP> -> DT ,, Count: 1
Rule: JJ -> 'index-fund', Count: 1
Rule: VBG -> 'leveraging', Count: 1
Rule: VBG -> 'owning', Count: 1
Rule: QP^<ADJP> -> JJR QP|<IN>^<ADJP>, Count: 1
Rule: QP|<IN>^<ADJP> -> IN CD, Count: 1
Rule: NP^<S> -> RB JJ, Count: 1
Rule: NP^<SINV> -> NNS, Count: 1
Rule: VP^<SINV> -> VB, Count: 1
Rule: JJS -> 'che

Rule: NP|<NNPS>^<PP> -> NNPS NN, Count: 1
Rule: CD -> '6.25', Count: 2
Rule: CD -> '7.74', Count: 2
Rule: CD -> '2029', Count: 3
Rule: CD -> '30.9', Count: 1
Rule: WHNP^<WHNP> -> DT NN, Count: 1
Rule: NP^<VP> -> CD NP|<,>^<VP>, Count: 1
Rule: CD -> '1992-1999', Count: 1
Rule: NP|<,>^<VP> -> , NP|<CD>^<VP>, Count: 2
Rule: NP|<CD>^<VP> -> CD NP|<CC>^<VP>, Count: 1
Rule: NP|<CC>^<VP> -> CC CD, Count: 1
Rule: CD -> '2017', Count: 2
Rule: JJ -> 'Fourth', Count: 1
Rule: NN -> 'serial', Count: 1
Rule: CD -> '7.40', Count: 2
Rule: CD -> '7.45', Count: 1
Rule: CD -> '7.50', Count: 1
Rule: CD -> '49.9', Count: 1
Rule: CD -> '2005', Count: 2
Rule: NNS -> 'Bonds', Count: 1
Rule: CD -> '7.65', Count: 1
Rule: JJ -> 'double-A', Count: 1
Rule: NN -> 'rating', Count: 6
Rule: NNP -> 'Heiwado', Count: 1
Rule: NNS -> 'Eurobonds', Count: 2
Rule: NP|<,> -> , NP|<PP>, Count: 12
Rule: JJ -> 'equity-purchase', Count: 1
Rule: VBG -> 'indicating', Count: 2
Rule: VP|<NP>^<NP> -> NP^<VP> VP|<,>^<NP>, Count: 1
Rule

We have a total number of rules:

In [109]:
n = sum(1 for c in counter_pcfg)
print(n)

16739


So if we look at the $6739$ most common ones we can already start seeing stuff:

In [112]:
counter_pcfg.most_common(n - 10000)

[(, -> ',', 3955),
 (DT -> 'the', 3268),
 (PP^<NP> -> IN NP^<PP>, 3191),
 (. -> '.', 3068),
 (S|<VP> -> VP^<S> ., 2414),
 (PP^<VP> -> IN NP^<PP>, 2139),
 (IN -> 'of', 1849),
 (TO -> 'to', 1697),
 (S^<SBAR> -> NP^<S> VP^<S>, 1485),
 (DT -> 'a', 1449),
 (S -> NP^<S> S|<VP>, 1410),
 (IN -> 'in', 1261),
 (CC -> 'and', 1229),
 (NP^<S> -> PRP, 1200),
 (NP^<PP> -> NP^<NP> PP^<NP>, 1024),
 (NP^<NP> -> DT NN, 1010),
 (VP^<S> -> TO VP^<VP>, 998),
 (S|<NP> -> NP^<S> S|<VP>, 948),
 (NP^<VP> -> NP^<NP> PP^<NP>, 828),
 (S^<VP> -> VP^<S>, 762),
 (IN -> 'for', 655),
 (VP^<VP> -> VB NP^<VP>, 651),
 (S|<,> -> , S|<NP>, 642),
 (`` -> '``', 622),
 ('' -> "''", 607),
 (NP^<NP> -> DT NP|<JJ>^<NP>, 595),
 (S^<SBAR> -> VP^<S>, 587),
 (POS -> "'s", 577),
 (VP^<S> -> MD VP^<VP>, 569),
 (DT -> 'The', 561),
 (VBZ -> 'is', 554),
 (ADVP^<VP> -> RB, 543),
 (S^<S> -> NP^<S> VP^<S>, 541),
 (SBAR^<VP> -> IN S^<SBAR>, 524),
 (NP^<S> -> NP^<NP> PP^<NP>, 511),
 (NP|<JJ>^<NP> -> JJ NN, 500),
 ($ -> '$', 477),
 (NP^<S> -> D

So we can get a sense that most rules are less frequent than 20 times, this threshold will cause us problems later, but we'll explain that soon.

Now we can rewrite our PCFG generator to fit that threshold:

In [128]:
def pcfg_cnf_learn_with_filtered_productions(treebank, n, additional_productions):
    trees = treebank.parsed_sents()[:n]
    trees_productions = []
    for tree in trees:
        new_tree = build_simplified_tree(tree)
        chomsky_normal_form(new_tree, factor='right', horzMarkov=1, vertMarkov=1, childChar='|', parentChar='^')
        productions = cnf_tree_to_productions(new_tree)
        trees_productions += [prod for prod in productions]
    c_cnf = Counter(trees_productions)
    n = sum(1 for c in c_cnf)
    filtered_productions = c_cnf.most_common(n - 10000)
    filtered_productions = [p for p, c in filtered_productions]
    trees_productions = filtered_productions + additional_productions
    pcfg_induced = induce_pcfg(Nonterminal('S'), trees_productions)
    return pcfg_induced

In [129]:
treebank = LazyCorpusLoader('treebank/combined', BracketParseCorpusReader, r'wsj_.*\.mrg')
pcfg_pos_filtered_training = pcfg_cnf_learn_with_filtered_productions(treebank, 3131, pos_productions)
v_pos_filtered_parser = nltk.ViterbiParser(pcfg_pos_filtered_training)

Now we can try parsing again:

In [130]:
parse_first_tree(test_set, v_pos_filtered_parser)

ValueError: Grammar does not cover some of the input words: "'10-year', '22\\\\/32'".

And unsurprisingly this didn't work.

The problems that rise from using this method are:
- We might delete needed rules so we won't be able to parse all sentences.
- We will reduce the preformance of our parser.

What happened here is we delelted low frequency rules, but such rules delete the rules for single words (that are usually less frequent, especially for uncommon word).
<br>We can overcome this by doing two things
- Skip all trees we can't parse (but we will lose a lot of trees this way).
- Add all the POS tags for the words in the test set - this makes our parser biased for our test set.
<br>We will show the results from both methods.

In [131]:
def parse_trees_skip(test_set, parser):
    skipped = 0
    l_precision = 0.0
    l_recall = 0.0
    u_precision = 0.0
    u_recall = 0.0
    total_parsed_trees = 0
    for tree in test_set:
        simplified = build_simplified_tree(tree)
        sentence = simplified.leaves()
        chomsky_normal_form(simplified, factor='right', horzMarkov=1, vertMarkov=1, childChar='|', parentChar='^')
        actual_tree_constituents = build_constituents(simplified)
        viterbi_parsed = parser.parse(sentence)
        try:
            for t in viterbi_parsed:
                total_parsed_trees += 1
                parsed_tree_constituents = build_constituents(t)
                l_precision += calc_labeled_precision(parsed_tree_constituents, actual_tree_constituents)
                l_recall += calc_labeled_recall(parsed_tree_constituents, actual_tree_constituents)
                u_precision += calc_unlabeled_precision(parsed_tree_constituents, actual_tree_constituents)
                u_recall += calc_unlabeled_recall(parsed_tree_constituents, actual_tree_constituents)
        except ValueError as e:
            skipped += 1
            continue
    return l_precision, l_recall, u_precision, u_recall, total_parsed_trees, skipped

In [ ]:
l_precision_skip, l_recall_skip, u_precision_skip, u_recall_skip, total_parsed_trees_skip, skipped = parse_trees_skip(test_set, v_pos_filtered_parser)
avg_pr_l_skip = l_precision_skip / float(total_parsed_trees_skip)
avg_rcl_l_skip = l_recall_skip / float(total_parsed_trees_skip) 
avg_pr_u_skip = u_precision_skip / float(total_parsed_trees_skip)
avg_rcl_u_skip = u_recall_skip / float(total_parsed_trees_skip)
print("Average Labeled Precision", avg_pr_l_skip)
print("Average Labeled Recall", avg_rcl_l_skip)
print("Average Unabeled Precision", avg_pr_u_skip)
print("Average Unlabeled Precision", avg_rcl_u_skip)
print("Skipped", skipped)

To test the preformance we will first arrange our test set by tree sentence length (to effect $n$) and then test the preformance of both parsers to the *'giving up'* point of our initial parser (which we found out is $n = $)